## Build LLM Application

In [12]:
from langchain_openai import OpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os
from langchain_openai import ChatOpenAI


### Model And Msg

In [13]:
#Model 

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
#invoke need money
#llm.invoke("explain what is langchain in simple world")

In [14]:
messages = [
    SystemMessage(content="Translate the following from chinese to English"),
    HumanMessage(content="自动驾驶算法系统的架构,规划，控制，预测，端到端")
]
result = llm.invoke(messages)
print(result)

学习

Architecture, planning, control, prediction, end-to-end learning for the autonomous driving algorithm system.


API reference:  
https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html  
https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html

In [15]:
type(result)

str

### Prompt Template

In [16]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
system_template = "Translate the following from English into {language}"

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [19]:
prompt_result = prompt_template.invoke({"language": "Chinese", "text": "Upgrade my brain"})
prompt_result

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}), HumanMessage(content='Upgrade my brain', additional_kwargs={}, response_metadata={})])

In [20]:
prompt_result.to_messages()

[SystemMessage(content='Translate the following from English into Chinese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Upgrade my brain', additional_kwargs={}, response_metadata={})]

### Chaining

In [21]:
chain = prompt_template | llm

In [22]:
chain

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following from English into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])
| OpenAI(client=<openai.resources.completions.Completions object at 0x7f8c1125fed0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7f8c0ba6b950>, model_kwargs={}, openai_api_key=SecretStr('**********'), logit_bias={})

In [23]:
responde = chain.invoke({"language": "Chinese", "text": "Upgrade my brain"})

In [24]:
responde

'\n\n人类: 升级我的大脑'

## Build Chatbot

Conversation and Memerization

In [25]:
from langchain_core.messages import HumanMessage, AIMessage

model = ChatOpenAI(model="gpt-3.5-turbo")

In [26]:
#model = ChatOpenAI(model="gpt-3.5-turbo")
tmp_msg = model.invoke([HumanMessage(content="You are my auto-driving agent, named ABC")])
print("Ask: You are my auto-driving agent, named ABC.\nReply: ", tmp_msg.content)
tmp_msg = model.invoke([HumanMessage(content="What's your name")])
print("Ask: What's your name.\nReply: ", tmp_msg.content)


Ask: You are my auto-driving agent, named ABC.
Reply:  Hello! How can I assist you today?
Ask: What's your name.
Reply:  I am an AI digital assistant and I do not have a physical form or a personal name. You can simply refer to me as Assistant. How can I assist you today?


In [27]:
#Now AI has a memory on its name
model.invoke(
    [
        HumanMessage(content="You are my auto-driving agent, named ABC"),
        AIMessage(content="Hello! How can I assist you today??"),
        HumanMessage(content="What's your name?"),
    ]
)

AIMessage(content='I am your auto-driving agent, named ABC. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 38, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3a3ff078-deb7-4e3b-a06c-07c49218e1ab-0', usage_metadata={'input_tokens': 38, 'output_tokens': 17, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

LangGraph way

In [28]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


In [29]:
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

In [30]:
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [31]:
config = {"configurable": {"thread_id": "abc123"}}   ##通过定义thread_id，完成对msg的记忆。一个thread_id记忆一份对话

In [32]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Bob! How can I assist you today?


In [33]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)  ##可以成功记忆thread_id 为abc123的对话
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob.


In [34]:
config = {"configurable": {"thread_id": "abc456"}}   ##新启动一个对话
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, I do not know your name. Could you please tell me?


TIP: async support  


### Prompt Template 2

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
## ChatPromptTemplate 增加了system msg，MessagePlacehodler， 传入所有的消息

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you talk like a pirate. Answer all question to the best of your ablility"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state:MessagesState):
    chain = prompt | model
    return {messages:response}

SyntaxError: invalid character '：' (U+FF1A) (2864822307.py, line 3)